In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score
import tensorflow as tf
import imageio
import keras
from random import choice
from scipy.ndimage import rotate, shift, zoom # For data augmentation


E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [3]:
root_dir = os.path.abspath('C:\\Users\\Rahul Jakhad\\Downloads\\digit recognition')


In [4]:
train = pd.read_csv(os.path.join(root_dir, 'Train', 'train.csv'))
test = pd.read_csv(os.path.join(root_dir, 'Test.csv'))

sample_submission = pd.read_csv(os.path.join(root_dir, 'Sample_Submission.csv'))

In [5]:
img_name = choice(train.filename)

In [6]:
temp = []
for img_name in train.filename:
    
    image_path = os.path.join(root_dir, 'Train', 'Images', 'train', img_name)
    img = imread(image_path, flatten=True)
    img = img.astype('float32')
    temp.append(img)
    
train_x = np.stack(temp)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  """


In [7]:
temp = []
for img_name in test.filename:
    image_path = os.path.join(root_dir, 'Train', 'Images', 'test', img_name)
    img = imread(image_path, flatten=True)
    img = img.astype('float32')
    temp.append(img)
    
test_x = np.stack(temp)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  after removing the cwd from sys.path.


In [8]:
train_x.shape

(49000, 28, 28)

In [9]:
test_x.shape

(21000, 28, 28)

In [10]:
train_x = train_x.reshape((49000, 28, 28,1))

In [11]:
train_x.shape

(49000, 28, 28, 1)

In [12]:
test_x = test_x.reshape((test_x.shape[0], 28, 28,1))

In [13]:
test_x.shape

(21000, 28, 28, 1)

In [14]:
train_x = train_x/255.0
test_x = test_x/255.0

In [15]:
split_size = int(train_x.shape[0]*0.7)

train_x, val_x = train_x[:split_size], train_x[split_size:]
train_y, val_y = train.label.values[:split_size], train.label.values[split_size:]

In [16]:
train_y = train_y.reshape((train_y.shape[0] , 1))
from keras.utils import to_categorical
train_y = to_categorical(train_y)
val_y = to_categorical(val_y)

print(train_y.shape , val_y.shape)

(34300, 10) (14700, 10)


In [18]:
import keras 
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization

In [20]:
input_shape = (None,train_x.shape[1] , train_x.shape[2] , train_x.shape[3] )

In [21]:
input_shape

(None, 28, 28, 1)

In [22]:
model = Sequential()

In [23]:
#Layer 1
#Conv Layer 1
model.add(Conv2D(filters = 6, 
                 kernel_size = 3, 
                 strides = 1, 
                 activation = 'relu', 
                 input_shape = (28,28,1)))

In [24]:
#Pooling layer 1
model.add(MaxPooling2D(pool_size = 2, strides = 2))

In [25]:
#Layer 2
#Conv Layer 2
model.add(Conv2D(filters = 16, 
                 kernel_size = 3,
                 strides = 1,
                 activation = 'relu',
                 input_shape = (14,14,6)))

In [26]:
#Pooling Layer 2
model.add(MaxPooling2D(pool_size = 2, strides = 2))

In [27]:
#Flatten
model.add(Flatten())

In [28]:
#Layer 3
#Fully connected layer 1
model.add(Dense(units = 120, activation = 'relu'))

In [29]:
#Layer 4
#Fully connected layer 2
model.add(Dense(units = 84, activation = 'relu'))

In [30]:
#Layer 5
#Output Layer
model.add(Dense(units = 10, activation = 'softmax'))

In [31]:
model.build(input_shape)

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 6)         60        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 16)        880       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_2 (Dense)              (None, 84)               

In [33]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [34]:
model.fit(train_x ,train_y , steps_per_epoch = 10, epochs = 42)


Epoch 1/42
10/10 [==============================] - 33s 3s/step - loss: 2.1446 - accuracy: 0.4214
Epoch 2/42
10/10 [==============================] - 31s 3s/step - loss: 1.5745 - accuracy: 0.6970
Epoch 3/42
10/10 [==============================] - 30s 3s/step - loss: 0.8835 - accuracy: 0.7976
Epoch 4/42
10/10 [==============================] - 30s 3s/step - loss: 0.4949 - accuracy: 0.8601
Epoch 5/42
10/10 [==============================] - 31s 3s/step - loss: 0.3444 - accuracy: 0.8986
Epoch 6/42
10/10 [==============================] - 37s 4s/step - loss: 0.2695 - accuracy: 0.9193
Epoch 7/42
10/10 [==============================] - 35s 3s/step - loss: 0.2201 - accuracy: 0.9330
Epoch 8/42
10/10 [==============================] - 37s 4s/step - loss: 0.1845 - accuracy: 0.9441
Epoch 9/42
10/10 [==============================] - 42s 4s/step - loss: 0.1577 - accuracy: 0.9521
Epoch 10/42
10/10 [==============================] - 37s 4s/step - loss: 0.1371 - accuracy: 0.9582
Epoch 11/42
10/10 

In [35]:
y_pred = model.predict(test_x)

In [36]:
#Converting one hot vectors to labels
labels = np.argmax(y_pred, axis = 1)

In [37]:
labels

array([4, 0, 9, ..., 6, 6, 2], dtype=int64)

In [38]:
sample_submission.label = labels

In [39]:
sample_submission.to_csv("prediction1.csv")